In [1]:
from __future__ import print_function
from __future__ import division
import os, sys
import numpy as np
import pandas as pd
from builtins import range
from sklearn.metrics import roc_auc_score
import librosa, librosa.display
import matplotlib.pyplot as plt
% matplotlib inline

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Flatten, Input, Reshape, Dropout, Permute
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras import backend as K
from keras.backend.tensorflow_backend import set_session

os.environ["CUDA_VISIBLE_DEVICES"]="0" # the number of the GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # percentage to be used
set_session(tf.Session(config=config))

from kapre.time_frequency import Melspectrogram
from global_config import *

Using TensorFlow backend.


Channel-last, i.e., (None, n_freq, n_time, n_ch)


In [2]:
def data_gen(df_subset, ys, is_shuffle, batch_size=20):
    """Data generator.
    df_subset: pandas dataframe, with rows subset
    ys: numpy arrays, N-by-8 one-hot-encoded labels
    is_shuffle: shuffle every batch if True.
    batch_size: integer, size of batch. len(df_subset) % batch_size should be 0.
    """
    n_data = len(df_subset)
    n_batch = n_data // batch_size
    if n_data % batch_size != 0:
        print("= WARNING =")
        print("  n_data % batch_size != 0 but this code does not assume it")
        print("  so the residual {} sample(s) will be ignored.".format(n_data % batch_size))

    while True:
        for batch_i in range(n_batch):
            if is_shuffle:
                batch_idxs = np.random.choice(n_data, batch_size, replace=False)
            else:
                batch_idxs = range(batch_i * batch_size, (batch_i + 1) * batch_size)

            src_batch = np.array([np.load(os.path.join(DIR_PEDAL_ONSET_NPY, df_subset.loc[df_subset.index[i]].filepath.split('.')[0]+'.npy')) for i in batch_idxs],
                                 dtype=K.floatx())
            src_batch = src_batch[:, np.newaxis, :]  # make (batch, N) to (batch, 1, N) for kapre compatible

            y_batch = np.array([ys[i] for i in batch_idxs],
                               dtype=K.floatx())
            
            yield src_batch, y_batch
        
        
def get_callbacks(name,patience):
    if not os.path.exists(DIR_SAVE_MODEL):
        os.makedirs(DIR_SAVE_MODEL)    
    early_stopper = keras.callbacks.EarlyStopping(patience=patience)
    model_saver = keras.callbacks.ModelCheckpoint(os.path.join(DIR_SAVE_MODEL,"{}_best_model.h5".format(name)),
                                                  save_best_only=True)
    weight_saver = keras.callbacks.ModelCheckpoint(os.path.join(DIR_SAVE_MODEL,"{}_best_weights.h5".format(name)),
                                                   save_best_only=True,
                                                   save_weights_only=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(DIR_SAVE_MODEL,"{}.log".format(name)))
    return [early_stopper, model_saver, weight_saver, csv_logger]

## Time-Invariant Models for Pedal-Onset Binary Classification

use raw-audio input, which is converted to melspectrogram using Kapre

In [3]:
def model_multi_kernel_shape(n_out, input_shape=ONSET_INPUT_SHAPE,
                             out_activation='softmax'):
    """

    Symbolic summary:
    > c2' - p2 - c2 - p2 - c2 - p2 - c2 - p3 - d1
    where c2' -> multiple kernel shapes

    Parameters
    ----------
        n_out: integer, number of output nodes
        input_shape: tuple, an input shape, which doesn't include batch-axis.
        out_activation: activation function on the output
    """
    audio_input = Input(shape=input_shape)

    x = Melspectrogram(n_dft=N_FFT, n_hop=HOP_LENGTH, sr=SR, n_mels=128, power_melgram=2.0, return_decibel_melgram=True)(audio_input)
    x = BatchNormalization(axis=channel_axis)(x)

    x1 = Conv2D(7, (20, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x2 = Conv2D(7, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x3 = Conv2D(7, (3, 20), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)

    x = Concatenate(axis=channel_axis)([x1, x2, x3])

    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((4, 4), padding='same')(x)
    x = Dropout(0.25)(x)

    x = GlobalAveragePooling2D()(x)

    out = Dense(n_out, activation=out_activation, kernel_regularizer=keras.regularizers.l2(reg_w))(x)

    model = Model(audio_input, out)

    return model


def model_crnn_icassp2017_choi(n_out, input_shape=ONSET_INPUT_SHAPE,
                               out_activation='softmax'):
    """A simplified model of 
    Convolutional Recurrent Neural Networks for Music Classification,
    K Choi, G Fazekas, M Sandler, K Choi, ICASSP, 2017, New Orleans, USA

    Symbolic summary:
    > c2 - p2 - c2 - p2 - c2 - p2 - c2 - p2 - r1 - r2 - d1

    Parameters
    ----------
        n_out: integer, number of output nodes
        input_shape: tuple, an input shape, which doesn't include batch-axis.
        out_activation: activation function on the output

    """

    audio_input = Input(shape=input_shape)

    x = Melspectrogram(n_dft=N_FFT, n_hop=HOP_LENGTH, sr=SR, n_mels=128, power_melgram=2.0, return_decibel_melgram=True)(audio_input)
    x = BatchNormalization(axis=channel_axis)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    x = Conv2D(21, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w))(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((4, 4), padding='same')(x)
    x = Dropout(0.25)(x)

    if K.image_dim_ordering() == 'channels_first':
        x = Permute((3, 1, 2))(x)

    x = Reshape((-1, 21))(x)

    # GRU block 1, 2, output
    x = GRU(41, return_sequences=True, name='gru1')(x)
    x = GRU(41, return_sequences=False, name='gru2')(x)
    x = Dropout(0.3)(x)

    out = Dense(n_out, activation=out_activation, kernel_regularizer=keras.regularizers.l2(reg_w))(x)

    model = Model(audio_input, out)

    return model


def model_conv3x3_ismir2016_choi(n_out, input_shape=ONSET_INPUT_SHAPE,
                                 out_activation='softmax'):
    """ A simplified model of 
    Automatic Tagging Using Deep Convolutional Neural Networks,
    K Choi, G Fazekas, M Sandler, ISMIR, 2016, New York, USA

    Symbolic summary:
    > c2 - p2 - c2 - p2 - c2 - p2 - c2 - p2 - c2 - p3 - d1

    Modifications: 
        * n_mels (96 -> 32)
        * n_channels (many -> [16, 24, 32, 40, 48])
        * remove dropout
        * maxpooling (irregular to fit the size -> all (2, 2))
        * add GlobalAveragePooling2D
    """

    model = Sequential()
    model.add(Melspectrogram(n_dft=N_FFT, n_hop=HOP_LENGTH, sr=SR, n_mels=128, power_melgram=2.0, 
                             return_decibel_melgram=True,
                             input_shape=input_shape))
    model.add(BatchNormalization(axis=channel_axis))

    model.add(Conv2D(10, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(15, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(15, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(20, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(20, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.25))

    model.add(GlobalAveragePooling2D())

    model.add(Dense(n_out, activation=out_activation, kernel_regularizer=keras.regularizers.l2(reg_w)))

    return model


def model_conv1d_icassp2014_sander(n_out, input_shape=ONSET_INPUT_SHAPE,
                                   out_activation='softmax'):
    """A simplified model of
    End-to-end learning for music audio,
    Sander Dieleman and Benjamin Schrauwen, ICASSP, 2014

    Symbolic summary:
    > c1 - p1 - c1 - p1 - c1 - p1 - p3 - d1

    Modifications: 
        * Add BatchNormalization
        * n_mels (128 -> 32)
        * n_layers (2 -> 3)
        * add GlobalAveragePooling2D

    Parameters
    ----------
        n_out: integer, number of output nodes
        input_shape: tuple, an input shape, which doesn't include batch-axis.
        out_activation: activation function on the output

    """

    model = Sequential()
    model.add(Melspectrogram(n_dft=N_FFT, n_hop=HOP_LENGTH, sr=SR, n_mels=128, power_melgram=2.0, 
                             return_decibel_melgram=True,input_shape=input_shape))
    model.add(Conv2D(30, (32, 4), padding='valid', kernel_regularizer=keras.regularizers.l2(reg_w)))  # (None, 16, 1, N)
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((1, 4), padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(30, (1, 4), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((1, 4), padding='same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(30, (1, 4), padding='same', kernel_regularizer=keras.regularizers.l2(reg_w)))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((1, 4), padding='same'))
    model.add(Dropout(0.25))

    model.add(GlobalAveragePooling2D())

    model.add(Dense(n_out, activation=out_activation, kernel_regularizer=keras.regularizers.l2(reg_w)))

    return model


In [4]:
reg_w = 1e-4
batch_size = 250
epochs = 40
patience = 10

In [4]:
dataset_name = 'pedal-onset_npydf_small.csv'
model_name = 'cnn3x3'
exp_name = 'small-onset_{}'.format(model_name)

print("-" * 60)
print("Beici: Welcome! Lets do something deep with {}.".format(dataset_name))
print("       I'm assuming you finished pre-processing.")
print("       We're gonna use {} model.".format(model_name))
csv_path = os.path.join(DIR_PEDAL_METADATA, dataset_name)

tracks = pd.read_csv(csv_path)
training = tracks.loc[tracks['category'] == 'train']
validation = tracks.loc[tracks['category'] == 'valid']
test = tracks.loc[tracks['category'] == 'test']

# print("Beici: We're loading and modifying label values.")
y_train = training.label.values
y_valid = validation.label.values
y_test = test.label.values

y_train = keras.utils.to_categorical(y_train, 2)
y_valid = keras.utils.to_categorical(y_valid, 2)
y_test = keras.utils.to_categorical(y_test, 2)

# callbacks
callbacks = get_callbacks(name=exp_name, patience=patience)
early_stopper, model_saver, weight_saver, csv_logger = callbacks

# print("Beici: Preparing data generators for training and validation...")
steps_per_epoch = len(y_train) // batch_size
gen_train = data_gen(training, y_train, True, batch_size=batch_size)
gen_valid = data_gen(validation, y_valid, False, batch_size=batch_size)
gen_test = data_gen(test, y_test, False, batch_size=batch_size)

print("Beici: Getting model...")
if model_name == 'multi_kernel':
    model = model_multi_kernel_shape(n_out=2)
elif model_name == 'crnn':
    model = model_crnn_icassp2017_choi(n_out=2)
elif model_name == 'cnn3x3':
    model = model_conv3x3_ismir2016_choi(n_out=2)
elif model_name == 'cnn1d':
    model = model_conv1d_icassp2014_sander(n_out=2)

# model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
# model.summary()

print("Beici: Starting to train...")
model.fit_generator(gen_train, steps_per_epoch, epochs=epochs,
                    callbacks=callbacks,
                    validation_data=gen_valid,
                    validation_steps=len(y_valid) // batch_size)

------------------------------------------------------------
Beici: Welcome! Lets do something deep with pedal-onset_npydf_small.csv.
       I'm assuming you finished pre-processing.
       We're gonna use cnn3x3 model.
Beici: Getting model...
Beici: Starting to train...
Epoch 1/40
280/280 [==============================] - 154s 549ms/step - loss: 0.6299 - acc: 0.6513 - val_loss: 0.6863 - val_acc: 0.5717
Epoch 2/40
280/280 [==============================] - 158s 565ms/step - loss: 0.5073 - acc: 0.7589 - val_loss: 0.6830 - val_acc: 0.6747
Epoch 3/40
280/280 [==============================] - 150s 535ms/step - loss: 0.4566 - acc: 0.7916 - val_loss: 1.0594 - val_acc: 0.5770
Epoch 4/40
280/280 [==============================] - 154s 550ms/step - loss: 0.4158 - acc: 0.8186 - val_loss: 0.7158 - val_acc: 0.6840
Epoch 5/40
280/280 [==============================] - 160s 571ms/step - loss: 0.3849 - acc: 0.8361 - val_loss: 1.1551 - val_acc: 0.6066
Epoch 6/40
280/280 [============================

In [6]:
print("Beici: Training is done. Loading the best weights...")
model.load_weights(os.path.join(DIR_SAVE_MODEL,"{}_best_weights.h5".format(exp_name)))

print("       Evaluating...")
scores = model.evaluate_generator(gen_test, len(y_test) // batch_size)
y_pred = model.predict_generator(gen_test, len(y_test) // batch_size)
auc = roc_auc_score(y_test, y_pred)

print("Beici: Done for {}!".format(model_name))
print("       test set loss: {}".format(scores[0]))
print("       test set accuracy: {}".format(scores[1]))
print("       test set auc: {}".format(auc))

Beici: Training is done. Loading the best weights...
       Evaluating...
Beici: Done for cnn3x3!
       test set loss: 0.193614187464
       test set accuracy: 0.933300049603
       test set auc: 0.98160964


In [5]:
dataset_name = 'pedal-onset_vd.csv'
model_name = 'multi_kernel'
exp_name = 'onset_{}'.format(model_name)
reg_w = 1e-4
batch_size = 250
epochs = 50
patience = 10

print("-" * 60)
print("Beici: Welcome! Lets do something deep with {}.".format(dataset_name))
print("       I'm assuming you finished pre-processing.")
print("       We're gonna use {} model.".format(model_name))
csv_path = os.path.join(DIR_PEDAL_METADATA, dataset_name)

tracks = pd.read_csv(csv_path)
training = tracks.loc[tracks['category'] == 'train']
validation = tracks.loc[tracks['category'] == 'valid']
test = tracks.loc[tracks['category'] == 'test']

# print("Beici: We're loading and modifying label values.")
y_train = training.label.values
y_valid = validation.label.values
y_test = test.label.values

y_train = keras.utils.to_categorical(y_train, 2)
y_valid = keras.utils.to_categorical(y_valid, 2)
y_test = keras.utils.to_categorical(y_test, 2)

# callbacks
callbacks = get_callbacks(name=exp_name, patience=patience)
early_stopper, model_saver, weight_saver, csv_logger = callbacks

# print("Beici: Preparing data generators for training and validation...")
steps_per_epoch = len(y_train) // batch_size
gen_train = data_gen(training, y_train, True, batch_size=batch_size)
gen_valid = data_gen(validation, y_valid, False, batch_size=batch_size)
gen_test = data_gen(test, y_test, False, batch_size=batch_size)

print("Beici: Getting model...")
if model_name == 'multi_kernel':
    model = model_multi_kernel_shape(n_out=2)
elif model_name == 'crnn':
    model = model_crnn_icassp2017_choi(n_out=2)
elif model_name == 'cnn3x3':
    model = model_conv3x3_ismir2016_choi(n_out=2)
elif model_name == 'cnn1d':
    model = model_conv1d_icassp2014_sander(n_out=2)

# model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
# model.summary()

print("Beici: Starting to train...")
model.fit_generator(gen_train, steps_per_epoch, epochs=epochs,
                    callbacks=callbacks,
                    validation_data=gen_valid,
                    validation_steps=len(y_valid) // batch_size)

------------------------------------------------------------
Beici: Welcome! Lets do something deep with pedal-onset_vd.csv.
       I'm assuming you finished pre-processing.
       We're gonna use multi_kernel model.
Beici: Getting model...
Beici: Starting to train...
= WARNING =
  n_data % batch_size != 0 but this code does not assume itEpoch 1/50

  so the residual 46 sample(s) will be ignored.
3571/3572 [============================>.] - ETA: 2s - loss: 0.3197 - acc: 0.8658= WARNING =
  n_data % batch_size != 0 but this code does not assume it
  so the residual 170 sample(s) will be ignored.
3572/3572 [==============================] - 12667s 4s/step - loss: 0.3197 - acc: 0.8659 - val_loss: 0.2132 - val_acc: 0.9237
Epoch 2/50
3572/3572 [==============================] - 8093s 2s/step - loss: 0.2147 - acc: 0.9197 - val_loss: 0.2178 - val_acc: 0.9189
Epoch 3/50
3572/3572 [==============================] - 4603s 1s/step - loss: 0.1978 - acc: 0.9270 - val_loss: 0.3236 - val_acc: 0.8569


In [ ]:
print("Beici: Training is done. Loading the best weights...")
model.load_weights(os.path.join(DIR_SAVE_MODEL,"{}_best_weights.h5".format(exp_name)))

print("       Evaluating...")
scores = model.evaluate_generator(gen_test, len(y_test) // batch_size)
y_pred = model.predict_generator(gen_test, len(y_test) // batch_size)

In [7]:
auc = roc_auc_score(y_test[:len(y_pred)], y_pred)

print("Beici: Done for {}!".format(model_name))
print("       test set loss: {}".format(scores[0]))
print("       test set accuracy: {}".format(scores[1]))
print("       test set auc: {}".format(auc))

Beici: Done for multi_kernel!
       test set loss: 0.160557327438
       test set accuracy: 0.943290049285
       test set auc: 0.9903246165
